In [5]:
import pandas as pd
import numpy as np

In [6]:
#synthetic dataset
import pandas as pd

# Create a sample dataset for learning pandas
data = {
    "Order ID": [101, 102, 103, 104, 105],
    "Product": ["Apple iPhone", "Samsung TV", "HP Laptop", "Office Chair", "Wooden Desk"],
    "Category": ["Electronics", "Electronics", "Electronics", "Furniture", "Furniture"],
    "Quantity": [2, 1, 1, 4, 2],
    "Price": [999, 499, 799, 120, 300],
    "Discount": [0.1, 0.15, 0.05, 0.2, 0.1],
    "Order Date": ["2024-01-01", "2024-01-03", "2024-01-05", "2024-01-10", "2024-01-15"],
    "Region": ["North", "South", "East", "West", "North"]
}

# Convert the data to a DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
file_path = "sample_sales_data.csv"
df.to_csv(file_path, index=False)

### Subset and slicing

In [7]:
df = pd.read_csv('sample_sales_data.csv')

In [10]:
df.head()

,Order ID,Product,Category,Quantity,Price,Discount,Order Date,Region
0,101,Apple iPhone,Electronics,2,999,0.10,2024-01-01,North
1,102,Samsung TV,Electronics,1,499,0.15,2024-01-03,South
2,103,HP Laptop,Electronics,1,799,0.05,2024-01-05,East
3,104,Office Chair,Furniture,4,120,0.20,2024-01-10,West
4,105,Wooden Desk,Furniture,2,300,0.10,2024-01-15,North


In [21]:
# get only region north
df[df.Region == "North"][["Product"]]

,Product
0,Apple iPhone
4,Wooden Desk


In [22]:
# get only region north as series
df[df.Region == "North"].Product

0    Apple iPhone
4     Wooden Desk
Name: Product, dtype: object

In [24]:
# use .loc 
df.loc[:, "Product"]

0    Apple iPhone
1      Samsung TV
2       HP Laptop
3    Office Chair
4     Wooden Desk
Name: Product, dtype: object

In [26]:
# use .loc as object
df.loc[:, ["Product"]]

,Product
0,Apple iPhone
1,Samsung TV
2,HP Laptop
3,Office Chair
4,Wooden Desk


In [28]:
df.loc[df.Region == "North", "Product"]

0    Apple iPhone
4     Wooden Desk
Name: Product, dtype: object

In [29]:
df.loc[df.Region == "North", ["Product"]]

,Product
0,Apple iPhone
4,Wooden Desk


In [31]:
# use .iloc
df.iloc[1:-1, 1]

1      Samsung TV
2       HP Laptop
3    Office Chair
Name: Product, dtype: object

In [33]:
# use .iloc as object
df.iloc[1:-1, [1]]

,Product
1,Samsung TV
2,HP Laptop
3,Office Chair


In [36]:
df.iloc[1:-1][["Product"]]

,Product
1,Samsung TV
2,HP Laptop
3,Office Chair


In [40]:
# conditional slicing
df.loc[(df.Region == "North") & (df.Category.isin(["Furniture", "Electronics"]))]

,Order ID,Product,Category,Quantity,Price,Discount,Order Date,Region
0,101,Apple iPhone,Electronics,2,999,0.1,2024-01-01,North
4,105,Wooden Desk,Furniture,2,300,0.1,2024-01-15,North


In [43]:
# conditional slicing
df.loc[(df.Region == "North") & (df.Category.isin(["Furniture", "Electronics"]))].iloc[[1, -1]]

,Order ID,Product,Category,Quantity,Price,Discount,Order Date,Region
4,105,Wooden Desk,Furniture,2,300,0.1,2024-01-15,North
4,105,Wooden Desk,Furniture,2,300,0.1,2024-01-15,North


#### Querying

In [47]:
df.query('Region == "North" and Category in ("Electronics", "Furniture")')

,Order ID,Product,Category,Quantity,Price,Discount,Order Date,Region
0,101,Apple iPhone,Electronics,2,999,0.1,2024-01-01,North
4,105,Wooden Desk,Furniture,2,300,0.1,2024-01-15,North


In [49]:
# use @ for variable
categories = ["Electronics", "Furniture"]
df.query('Region == "North" and Category in @categories')

,Order ID,Product,Category,Quantity,Price,Discount,Order Date,Region
0,101,Apple iPhone,Electronics,2,999,0.1,2024-01-01,North
4,105,Wooden Desk,Furniture,2,300,0.1,2024-01-15,North


#### Grouping

In [52]:
df.groupby("Region")[["Quantity"]].sum().reset_index()

,Region,Quantity
0,East,1
1,North,4
2,South,1
3,West,4


In [54]:
df.groupby("Category")[["Quantity"]].mean().reset_index()

,Category,Quantity
0,Electronics,1.333333
1,Furniture,3.000000


In [56]:
# group by multiple columns
df.groupby(["Category", "Region"])[["Quantity"]].sum().reset_index()

,Category,Region,Quantity
0,Electronics,East,1
1,Electronics,North,2
2,Electronics,South,1
3,Furniture,North,2
4,Furniture,West,4


#### Aggregation

In [58]:
# simple aggregation
df.groupby("Category").agg(
    Total_qty=("Quantity", "sum"),
    Avg_price=("Price", "median")
).reset_index()

,Category,Total_qty,Avg_price
0,Electronics,4,799.0
1,Furniture,6,210.0


In [64]:
# custom aggregation column
def range_func(item):
    return item.max() - item.min()

df.groupby("Category").agg(
    range_price=("Price", range_func)
).reset_index()

,Category,range_price
0,Electronics,500
1,Furniture,180


#### Pivot table

In [66]:
"""
With one value to pivot.

values="Quantity": The column to aggregate.
index="Region": Rows are grouped by the Region column.
columns="Category": Creates separate columns for each unique value in the Category column.
aggfunc="sum": Aggregates values using the sum function.
fill_value=0: Fills missing values with 0.
"""
df.pivot_table(
    values="Quantity",
    index="Region",
    columns="Category",
    aggfunc="sum",
    fill_value=0
)

Category,Electronics,Furniture
Region,,
East,1,0
North,2,2
South,1,0
West,0,4


In [68]:
"""
With multiple values to pivot.

values="Quantity": The column to aggregate.
index="Region": Rows are grouped by the Region column.
columns="Category": Creates separate columns for each unique value in the Category column.
aggfunc="sum": Aggregates values using the sum function.
fill_value=0: Fills missing values with 0.
"""
df.pivot_table(
    values=["Quantity", "Price"],
    index="Region",
    columns="Category",
    aggfunc="sum",
    fill_value=0
)

Price              Quantity          
Category Electronics Furniture Electronics Furniture
Region                                              
East             799         0           1         0
North            999       300           2         2
South            499         0           1         0
West               0       120           0         4

#### apply

In [72]:
"""
The apply function allows you to apply a function (or lambda) to either rows or columns.
row is axis=0 
"""
df[["Price"]].apply(lambda x: x * 0.1, axis=0)

,Price
0,99.9
1,49.9
2,79.9
3,12.0
4,30.0


#### map

In [74]:
"""
The map function works element-wise on a Series. It’s faster and simpler than apply when working with a single column.
therefore no need axis
"""
df[["Price"]].map(lambda x: x * 0.1)

,Price
0,99.9
1,49.9
2,79.9
3,12.0
4,30.0


#### applymap

In [75]:
"""
The applymap function applies a function to every element in a DataFrame (used for DataFrames, not Series).
"""

'\nThe applymap function applies a function to every element in a DataFrame (used for DataFrames, not Series).\n'

In [77]:
df[["Price"]].applymap(lambda x: x * 0.1)

/var/folders/4d/c92r7j0x63v8v0t9zqv4cxpr0000gn/T/ipykernel_2180/1884092189.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[["Price"]].applymap(lambda x: x * 0.1)


,Price
0,99.9
1,49.9
2,79.9
3,12.0
4,30.0


#### complex lambda

In [78]:

data = {
    "Order ID": [101, 102, 103, 104, 105],
    "Product": ["Apple iPhone", "Samsung TV", "HP Laptop", "Office Chair", "Wooden Desk"],
    "Category": ["Electronics", "Electronics", "Electronics", "Furniture", "Furniture"],
    "Quantity": [2, 1, 1, 4, 2],
    "Price": [999, 499, 799, 120, 300],
    "Discount": [0.1, 0.15, 0.05, 0.2, 0.1],
    "Order Date": ["2024-01-01", "2024-01-03", "2024-01-05", "2024-01-10", "2024-01-15"],
    "Region": ["North", "South", "East", "West", "North"]
}

df = pd.DataFrame(data)

In [79]:
df.head()

,Order ID,Product,Category,Quantity,Price,Discount,Order Date,Region
0,101,Apple iPhone,Electronics,2,999,0.10,2024-01-01,North
1,102,Samsung TV,Electronics,1,499,0.15,2024-01-03,South
2,103,HP Laptop,Electronics,1,799,0.05,2024-01-05,East
3,104,Office Chair,Furniture,4,120,0.20,2024-01-10,West
4,105,Wooden Desk,Furniture,2,300,0.10,2024-01-15,North


In [87]:
def calculate_final_cost(row, tax_rate, shipping_cost):
    # row is the row from dataframe
    base_cost = row["Quantity"] * row["Price"]
    discount_amount = base_cost * row["Discount"]
    discount_cost = base_cost - discount_amount
    tax_amount = discount_cost* tax_rate
    final_cost = discount_cost + tax_amount + shipping_cost

    return final_cost

# arguments for the function
tax_rate = 0.05
shipping_cost = 20

# apply upon each row
df["Final Cost"] = df.apply(lambda row: calculate_final_cost(row, tax_rate, shipping_cost), axis=1)
df.head(-1)

,Order ID,Product,Category,Quantity,Price,Discount,Order Date,Region,Final Cost
0,101,Apple iPhone,Electronics,2,999,0.10,2024-01-01,North,1908.1100
1,102,Samsung TV,Electronics,1,499,0.15,2024-01-03,South,465.3575
2,103,HP Laptop,Electronics,1,799,0.05,2024-01-05,East,817.0025
3,104,Office Chair,Furniture,4,120,0.20,2024-01-10,West,423.2000
